In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Glucocorticoid_Sensitivity/GSE65645'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"LncRNA Expression Discriminates Karyotype and Predicts Survival in B-lymphoblastic Leukemia (Agilent)"
!Series_summary	"Long non-coding RNAs (lncRNAs) have been found to play a role in gene regulation with dysregulated expression in various cancers. The precise role that lncRNA expression plays in the pathogenesis of B-acute lymphoblastic leukemia (B-ALL) is unknown. Therefore, unbiased microarray profiling was performed on human B-ALL specimens and it was determined that lncRNA expression correlates with cytogenetic abnormalities, which was confirmed by RT-qPCR in a large set of B-ALL cases. Importantly, high expression of BALR-2 correlated with poor overall survival and diminished response to prednisone treatment. In line with a function for this lncRNA in regulating cell survival, BALR-2 knockdown led to reduced proliferation, increased apoptosis, and increased sensitivity to prednisolone treatment. Conversely, overexpression of BALR-2 led to i

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Check if gene expression data is available
is_gene_available = True  # inferred from the microarray profiling for gene expression

# Data availability and identification of keys
sample_characteristics_dict = {
    0: ['sample_type: bone marrow'],
    1: ['translocation: TEL_AML1', 'translocation: E2A_PBX1', 'translocation: MLL']
}

# Assuming human data is not labeled for Glucocorticoid_Sensitivity, age, and gender directly in the sample characteristics
trait_row = None
age_row = None
gender_row = None

# Data type conversion functions
def convert_trait(trait_value):
    value = trait_value.split(':')[-1].strip()
    return float(value) if value.isdigit() else None

def convert_age(age_value):
    value = age_value.split(':')[-1].strip()
    return float(value) if value.isdigit() else None

def convert_gender(gender_value):
    value = gender_value.split(':')[-1].strip().lower()
    if value == 'male':
        return 1
    elif value == 'female':
        return 0
    return None

# Save cohort information
save_cohort_info('GSE65645', './preprocessed/Glucocorticoid_Sensitivity/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical feature extraction skipped since trait_row is None
